In [1]:
import pyodbc
import sqlalchemy as sal
from sqlalchemy import create_engine
import pandas as pd

### Establish the SQL Connection

In [2]:
server_name = 'GVX0WMERC01P'
database_name = 'GC_PRICE'
engine = sal.create_engine('mssql+pyodbc://GVX0WMERC01P/GC_PRICE?driver=SQL Server?Trusted_Connection=yes')
print(engine)

Engine(mssql+pyodbc://GVX0WMERC01P/GC_PRICE?driver=SQL+Server)


In [3]:
# establishing the connection to the databse using engine as an interface
conn = engine.connect()
# cnxn = engine.connect()

In [4]:
# print(engine.table_names())

### Import the Data into Dataframes

In [5]:
# checking whether the connection was actually established by selecting and displaying contents of table from the database
# query = "select [Item Code], [LIR Description] from [Lookup item code]"
# result = engine.execute(query)
# for row in result:
#     print (row)
# result.close()

ausa_promo_history_query = """
WITH Promo_History AS (
  SELECT
    [w/e],
    [LIR Description],
    --[Competitor],
    [AUSA EDLP],
	[AUSA Actual]
    --[Comp EDLP],
    --[Comp Actual]
  FROM
    [GC_PRICE].[dbo].[LJ - Indexes - Item Index Data Master] Indices
    JOIN (
      SELECT
        [str_num]
      FROM
        [GC_PRICE].[dbo].[Lookup store]
      WHERE
        [AUSA Banner] = 'giant foods'
    ) AS Stores ON Stores.STR_NUM = Indices.Store
    JOIN (
      SELECT
        [item code],
        [lir description]
      FROM
        [GC_PRICE].[dbo].[lookup item code]
    ) AS Items ON Items.[Item Code] = Indices.[Item Code]
    --JOIN (
    --  SELECT
    --    [str_shrt_nm] Brand,
    --    [str_num]
    --  FROM
    --    [GC_PRICE].[dbo].[lookup competitor]
    --  WHERE
    --    [str_shrt_nm] IN ('Wegmans', 'Walmart', 'Safeway', 'Harris Teeter')
    --) AS Competitors ON Competitors.[str_num] = Indices.Competitor --where [w/e] > '2022-01-01' and [w/e] < '2022-12-31'
  WHERE
    [w/e] > dateadd(week, -52, getdate())
  GROUP BY
    [w/e]
    ,[LIR Description]
    --[Competitor],
    ,[AUSA EDLP]
    ,[AUSA Actual]
    --,[Comp EDLP]
    --,[Comp Actual] --Order by [LIR Description], [Competitor] asc
)
SELECT
  count(*) Weeks,
  [LIR Description],
  --[Competitor],
  [AUSA EDLP],
  [AUSA Actual]
  --[Comp EDLP],
  --[Comp Actual]
FROM
  Promo_History
GROUP BY
  [LIR Description],
  --[Competitor],
  --Brand --,
  [AUSA EDLP],
  [AUSA Actual]
  --[Comp EDLP],
  --[Comp Actual]
ORDER BY
  [LIR Description]
  """
# result = engine.execute(ausa_promo_history_query)
# for row in result:
#     print (row)
# result.close()

In [ ]:
ausa_promo_history = pd.read_sql(ausa_promo_history_query, conn)
ausa_promo_history.head()

In [ ]:
comp_promo_history_query = 
"""
WITH Promo_History AS (
  SELECT
    [w/e],
    [LIR Description],
    [Competitor],
    Competitors.Brand --,[AUSA EDLP]
    --,[AUSA Actual]
,
    [Comp EDLP],
    [Comp Actual]
  FROM
    [GC_PRICE].[dbo].[LJ - Indexes - Item Index Data Master] Indices
    JOIN (
      SELECT
        [str_num]
      FROM
        [GC_PRICE].[dbo].[Lookup store]
      WHERE
        [AUSA Banner] = 'giant foods'
    ) AS Stores ON Stores.STR_NUM = Indices.Store
    JOIN (
      SELECT
        [item code],
        [lir description]
      FROM
        [GC_PRICE].[dbo].[lookup item code]
    ) AS Items ON Items.[Item Code] = Indices.[Item Code]
    JOIN (
      SELECT
        [str_shrt_nm] Brand,
        [str_num]
      FROM
        [GC_PRICE].[dbo].[lookup competitor]
      WHERE
        [str_shrt_nm] IN ('Wegmans', 'Walmart', 'Safeway', 'Harris Teeter')
    ) AS Competitors ON Competitors.[str_num] = Indices.Competitor --where [w/e] > '2022-01-01' and [w/e] < '2022-12-31'
  WHERE
    [w/e] > dateadd(week, -52, getdate())
  GROUP BY
    [w/e],
    [LIR Description],
    [Competitor],
    Competitors.Brand --,[AUSA EDLP]
    --,[AUSA Actual]
,
    [Comp EDLP],
    [Comp Actual] --Order by [LIR Description], [Competitor] asc
)
SELECT
  count(*) Weeks,
  [LIR Description],
  [Competitor],
  Brand --,[AUSA EDLP]
  --,[AUSA Actual]
,
  [Comp EDLP],
  [Comp Actual]
FROM
  Promo_History
GROUP BY
  [LIR Description],
  [Competitor],
  Brand --,[AUSA EDLP]
  --,[AUSA Actual]
,
  [Comp EDLP],
  [Comp Actual]
ORDER BY
  [LIR Description],
  Brand
"""
comp_promo_history = pd.read_sql(comp_promo_history_query, conn)
comp_promo_history.head()